# Wave 2 Analysis - 13 Year Old Cohort

### 2 Datasets for Wave 2:
- Cohort dataset (details about each student, e.g. demographics, academic scores, etc)
- Time Use dataset (details of the time use diary completed by students)

## Import Required Packages

In [1]:
import math
import statistics
import pandas as pd
import numpy as np
import warnings
from pandas.core.common import SettingWithCopyWarning
from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.weightstats import ztest as ztest

# Silence copy warning & allow all columns and rows to be seen
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Import Raw Data - Cohort Dataset

In [2]:
# Specify which columns we want to use (using the data dictionaries)
required_cols = ['id', 'wgt_13yr', 'gross_13yr', 'nals', 'p2sexw2']

# Read in data
cohort_data = pd.read_csv('0020-02 GUI Child Cohort Wave 2/0020-02 GUI Child Cohort Wave 2_Data/13 Year Cohort Data/CSV/GUI Data_ChildCohortWave2.csv', index_col= 'id', usecols= required_cols)

# Check the number of rows and columns
cohort_data.shape

(7525, 4)

## Exploratory Data Analysis - Cohort Dataset

- Check the datatypes of each column
- Describe the data - min, max, mean, stdev, etc
- View the first 10 rows of the data

In [3]:
cohort_data.dtypes

wgt_13yr      float64
gross_13yr    float64
p2sexw2         int64
nals           object
dtype: object

In [4]:
cohort_data.describe()

,wgt_13yr,gross_13yr,p2sexw2
count,7525.000000,7525.000000,7525.000000
mean,0.999995,7.414710,1.510698
std,0.958504,7.107068,0.499919
min,0.126819,0.940334,1.000000
25%,0.443806,3.290709,1.000000
50%,0.705358,5.230055,2.000000
75%,1.183765,8.777322,2.000000
max,14.430778,107.000623,2.000000


*Below output is removed from Git as access to the data is to be requested from Growing up in Ireland.*

In [ ]:
cohort_data.head(10)

## Clean Raw Cohort Dataset

- Standardise the format of all empty/missing values to use Python NULL so we can use Python .dropna(), .isna(), etc
- Swap '999' maths score value for a Python NULL value
- Drop any rows where the maths score is missing
- Set the datatype of the maths score as a decimal
- Create 2 new columns - the scaled maths score (1-100) and the percentile rank using the scaled maths score

In [5]:
# NULL values are filled with whitespace, replace with Python numpy NULL value
cohort_data = cohort_data.replace(r'^\s*$', np.nan, regex=True)

# 37 records have 999 as the maths score, treat this as a missing value also
cohort_data['nals'] = cohort_data['nals'].replace('999', np.nan)

# Drop any students who are missing a maths score
cohort_data = cohort_data[cohort_data.nals.isna() == False]

print(f'Shape of dataframe after dropping students missing a maths score: {cohort_data.shape}')

# correct datatype from object to decimal(numeric)
cohort_data.nals = cohort_data.nals.astype('float64')

# Scale Maths logit scores to 1-100 using MinMaxScaler
scaler = MinMaxScaler(feature_range=(1,100))

cohort_data['SCALED_MATHSLS'] = scaler.fit_transform(cohort_data[['nals']])

# Calculate Percentile rank for students with maths score
cohort_data['MATHSLS_PERCENTILE_RANK'] = cohort_data['SCALED_MATHSLS'].rank(pct=True) * 100

print(f'Shape of dataframe after adding 2 new columns: {cohort_data.shape}')

Shape of dataframe after dropping students missing a maths score: (7111, 4)
Shape of dataframe after adding 2 new columns: (7111, 6)


### View first 10 rows after data cleaning

*Below output is removed from Git as access to the data is to be requested from Growing up in Ireland.*

In [ ]:
cohort_data.head(10)

## Import Raw Data - Time Use Dataset

In [6]:
timeuse_data = pd.read_csv('0020-02 GUI Child Cohort Wave 2/0020-02 GUI Child Cohort Wave 2_Data/Time Use Data/GUI Data_ChildCohortWave2_TimeUse.csv', index_col= 'ID')

timeuse_data.shape

(5023, 301)

## Exploratory Data Analysis - Time Use Dataset

- Check the datatypes of each column
- Describe the data - min, max, mean, stdev, etc
- View the first 10 rows of the data

In [7]:
timeuse_data.dtypes

wgttime13yr      float64
grosstime13yr    float64
bDiaryDay          int64
bDiaryDat         object
bdiarymonth        int64
bweekend           int64
bterm              int64
bt00_1_A1          int64
bt00_1_A2         object
bt00_1_A3         object
bt00_2_A1          int64
bt00_2_A2         object
bt00_2_A3         object
bt00_3_A1          int64
bt00_3_A2         object
bt00_3_A3         object
bt00_4_A1          int64
bt00_4_A2         object
bt00_4_A3         object
bt01_1_A1          int64
bt01_1_A2         object
bt01_1_A3         object
bt01_2_A1          int64
bt01_2_A2         object
bt01_2_A3         object
bt01_3_A1          int64
bt01_3_A2         object
bt01_3_A3         object
bt01_4_A1          int64
bt01_4_A2         object
bt01_4_A3         object
bt02_1_A1          int64
bt02_1_A2         object
bt02_1_A3         object
bt02_2_A1          int64
bt02_2_A2         object
bt02_2_A3         object
bt02_3_A1          int64
bt02_3_A2         object
bt02_3_A3         object


In [8]:
timeuse_data.describe()

,wgttime13yr,grosstime13yr,bDiaryDay,bdiarymonth,bweekend,bterm,bt00_1_A1,bt00_2_A1,bt00_3_A1,bt00_4_A1,bt01_1_A1,bt01_2_A1,bt01_3_A1,bt01_4_A1,bt02_1_A1,bt02_2_A1,bt02_3_A1,bt02_4_A1,bt03_1_A1,bt03_2_A1,bt03_3_A1,bt03_4_A1,bt04_1_A1,bt04_2_A1,bt04_3_A1,bt04_4_A1,bt05_1_A1,bt05_2_A1,bt05_3_A1,bt05_4_A1,bt06_1_A1,bt06_2_A1,bt06_3_A1,bt06_4_A1,bt07_1_A1,bt07_2_A1,bt07_3_A1,bt07_4_A1,bt08_1_A1,bt08_2_A1,bt08_3_A1,bt08_4_A1,bt09_1_A1,bt09_2_A1,bt09_3_A1,bt09_4_A1,bt10_1_A1,bt10_2_A1,bt10_3_A1,bt10_4_A1,bt11_1_A1,bt11_2_A1,bt11_3_A1,bt11_4_A1,bt12_1_P1,bt12_2_P1,bt12_3_P1,bt12_4_P1,bt01_1_P1,bt01_2_P1,bt01_3_P1,bt01_4_P1,bt02_1_P1,bt02_2_P1,bt02_3_P1,bt02_4_P1,bt03_1_P1,bt03_2_P1,bt03_3_P1,bt03_4_P1,bt04_1_P1,bt04_2_P1,bt04_3_P1,bt04_4_P1,bt05_1_P1,bt05_2_P1,bt05_3_P1,bt05_4_P1,bt06_1_P1,bt06_2_P1,bt06_3_P1,bt06_4_P1,bt07_1_P1,bt07_2_P1,bt07_3_P1,bt07_4_P1,bt08_1_P1,bt08_2_P1,bt08_3_P1,bt08_4_P1,bt09_1_P1,bt09_2_P1,bt09_3_P1,bt09_4_P1,bt10_1_P1,bt10_2_P1,bt10_3_P1,bt10_4_P1,bt11_1_P1,bt11_2_P1,bt11_3_P1,bt11_4_P1,bt2_1,bt2_2,bt2_3,bT3,bT5
count,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.00000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000,5023.000000
mean,0.999994,11.108035,3.890305,7.948039,1.265777,1.097949,1.127812,1.125622,1.100338,1.091181,1.059327,1.051961,1.040215,1.037627,1.033048,1.030261,1.028270,1.025682,1.016325,1.014533,1.014533,1.014533,1.012741,1.011348,1.005574,1.005773,1.007167,1.009357,1.015529,1.023691,1.035039,1.043002,1.070874,1.152698,1.358551,1.583317,2.142942,2.811467,3.220983,3.830978,4.586900,4.699383,4.452518,4.643440,4.928927,5.025085,5.401354,5.797133,6.223771,6.423253,6.798726,7.146128,7.325503,7.392395,7.411308,7.527175,7.550269,7.710930,7.093171,7.178180,7.333068,7.441569,7.288473,7.490743,7.537926,7.713319,7.52180,7.650010,8.100338,8.272148,9.373681,10.106908,10.479395,10.724667,10.019311,10.374079,10.548676,10.731834,10.091977,10.411905,10.735815,10.929325,10.688831,10.822815,11.484571,11.683854,11.974517,12.471830,12.858053,13.094565,12.576747,12.590484,11.629305,11.037428,8.542504,7.978698,5.996815,5.207645,3.532550,3.081824,2.486164,2.363528,1.505674,3.224766,3.330281,2.994426,2.310372
std,1.068323,11.867037,2.001123,3.885084,0.441790,0.297276,1.257074,1.249272,1.106390,1.062769,0.839857,0.791411,0.663354,0.643858,0.675938,0.646561,0.631066,0.599961,0.509695,0.478935,0.478935,0.478935,0.451817,0.443178,0.200479,0.200970,0.213886,0.226025,0.357763,0.492108,0.581788,0.627701,0.788049,2.192266,2.721408,3.161424,5.283582,7.899892,7.536143,8.758923,10.073069,9.228288,4.697255,5.128077,5.721911,5.757561,6.588745,7.861863,9.109805,9.400257,9.656872,10.357227,10.451047,10.452769,10.046767,10.354067,10.182182,10.658247,8.513002,8.863287,9.317509,9.417700,8.026080,8.467561,8.577899,8.903965,7.62256,7.982068,9.543894,9.933701,12.991125,14.403385,14.55477

*Below output is removed from Git as access to the data is to be requested from Growing up in Ireland.*

In [ ]:
timeuse_data.head()

## Clean Raw Time Use Dataset

- Standardise the format of all empty/missing values to use Python NULL so we can use Python .dropna(), .isna(), etc


In [9]:
# NULL values are filled with whitespace, replace with Python numpy NULL value
timeuse_data = timeuse_data.replace(r'^\s*$', np.nan, regex=True)

# Check % of NULLs across each column, e.g. 50.0 = 50% of values in this column are NULL
timeuse_data.isnull().mean() * 100

wgttime13yr        0.000000
grosstime13yr      0.000000
bDiaryDay          0.000000
bDiaryDat          2.508461
bdiarymonth        0.000000
bweekend           0.000000
bterm              0.000000
bt00_1_A1          0.000000
bt00_1_A2         99.860641
bt00_1_A3        100.000000
bt00_2_A1          0.000000
bt00_2_A2         99.860641
bt00_2_A3         99.980092
bt00_3_A1          0.000000
bt00_3_A2         99.880549
bt00_3_A3         99.980092
bt00_4_A1          0.000000
bt00_4_A2         99.880549
bt00_4_A3         99.980092
bt01_1_A1          0.000000
bt01_1_A2         99.960183
bt01_1_A3        100.000000
bt01_2_A1          0.000000
bt01_2_A2         99.960183
bt01_2_A3        100.000000
bt01_3_A1          0.000000
bt01_3_A2         99.960183
bt01_3_A3        100.000000
bt01_4_A1          0.000000
bt01_4_A2         99.940275
bt01_4_A3        100.000000
bt02_1_A1          0.000000
bt02_1_A2         99.960183
bt02_1_A3        100.000000
bt02_2_A1          0.000000
bt02_2_A2         99

The format of the time use data is as follows:

sample column name = **bt00_1_A1**

This decoded = time 00:00, block 1/4 (i.e. first 15mins of 00hrs), A1 = activity 1

In [10]:
# get all column names that begin with 'bt'
timeuse_cols = [col for col in timeuse_data if col.startswith('bt')]

# specify columns we want to remove
cols_to_remove = ['bterm', 'bt2_1', 'bt2_2', 'bt2_3']

# final columns to be used is list 1 - list2
timeuse_cols = list(set(timeuse_cols) - set(cols_to_remove))

# sort the columns
timeuse_cols.sort()

## Feature Engineering Time Use Dataset

The following features are added to the time use dataset:
- The number of 15minute intervals each student spent gaming (using the data dictionary, gaming = activity '12')
- The total number of minutes each student spent gaming (i.e. number of intervals * 15 minutes)
- An indicator as to whether they gamed or not (for later use in filtering and visualisations)

In [11]:
# Gaming (activity 12)
timeuse_data_temp_12 = timeuse_data[timeuse_cols]

# Replace all activities recorded other than 'Gaming' (12) with NULLs
timeuse_data_temp_12[timeuse_data_temp_12 != 12] = np.nan

In [12]:
# Calculate count of 15min slots flagged as gaming per student ID using 'count'
timeuse_data_temp_12['CNT_15M_INTERVALS_COMPGAMES'] = timeuse_data_temp_12.count(axis=1)

# Calculate total time spent by multiplying the count of slots * 15 (as each slot = 15mins)
timeuse_data_temp_12['TOTAL_MINS_COMPGAMES'] = timeuse_data_temp_12.CNT_15M_INTERVALS_COMPGAMES * 15

# Add 2 new columns back to original dataset
timeuse_data = timeuse_data.join(timeuse_data_temp_12[['CNT_15M_INTERVALS_COMPGAMES', 'TOTAL_MINS_COMPGAMES']])

# Create a gaming indicator (1 = Yes, 0 = No)
timeuse_data['COMP_GAMING_IND'] = np.where(timeuse_data['CNT_15M_INTERVALS_COMPGAMES'] > 0, 1, 0)

## Merge Cohort Dataset & Time Use Dataset

In [13]:
# rename index so bothn datasets can be joined on the same column
cohort_data.index.set_names('ID', inplace=True)

# join both datasets using the common 'ID' column for each student
wave2_merged = pd.merge(cohort_data, timeuse_data, how='left', on='ID')

wave2_merged.shape

(7111, 310)

## After merge, drop any students missing a Time Use diary

In [14]:
# Drop anyone present in cohort data with a maths score who did not complete a time-use diary
wave2_merged = wave2_merged[wave2_merged.bDiaryDay.isna() == False]

print (f'Final number of rows and columns: {wave2_merged.shape}')

Final number of rows and columns: (4895, 310)


## Exploring the clean merged dataset 

In [15]:
# See breakdown of gaming vs no gaming
wave2_merged.COMP_GAMING_IND.value_counts()

0.0    3656
1.0    1239
Name: COMP_GAMING_IND, dtype: int64

## Statistical Testing

Compare the scaled maths score between those who game and didn't game.

**Null hypothesis:** The maths scores for students who did game = the maths scores for students who didn't game.

**Alternative hypothesis:** The maths scores for students who did game > the maths scores for students who didn't game.

In [16]:
# Split dataset into 2 groups, those who gamed and those who didn't
students_who_gamed = wave2_merged[wave2_merged.COMP_GAMING_IND == 1]
students_who_didnt_game = wave2_merged[wave2_merged.COMP_GAMING_IND == 0]

In [17]:
# perform two sample z-test
zscore, pvalue = ztest(students_who_gamed.SCALED_MATHSLS, students_who_didnt_game.SCALED_MATHSLS, value=0, alternative='larger')

confidence_threshold = 0.05 # 95% (1-0.05)

print('Z Score\t:', zscore)
print('P Value\t:', pvalue)

if (1 - pvalue < confidence_threshold):
    print(1 - pvalue)
    print('Null hypothesis is accepted!')
else:
    print(1 - pvalue)
    print('Null hypothesis is rejected. \nAlternate hypothesis is accepted!')

Z Score	: 5.845688044425868
P Value	: 2.5223934141481314e-09
0.9999999974776066
Null hypothesis is rejected. 
Alternate hypothesis is accepted!


In [18]:
# Add ID back as a column by resetting the index
wave2_merged.reset_index(inplace=True)

# Save clean merged dataset ready for Tableau to import
wave2_merged.to_csv('child_wave_2_merged.csv', index=False)

## Merging Clean Wave 1 and Wave 2 data

To identify students present in both waves (i.e. same student at different ages).

In [19]:
wave1 = pd.read_csv('child_wave_1_merged.csv')
wave2 = wave2_merged.copy()
wave2.reset_index(inplace=True)

print(f'Wave 1 shape: {wave1.shape}')

print(f'Wave 2 shape: {wave2.shape}')

Wave 1 shape: (6159, 515)
Wave 2 shape: (4895, 312)


In [20]:
wave_1_and_2_merged = pd.merge(wave1, wave2, on='ID')

wave_1_and_2_merged.shape

(4184, 826)

In [21]:
wave_1_and_2_merged.to_csv('child_wave_1_and_2_merged.csv', index=False)